<a href="https://colab.research.google.com/github/Sydney-Kelly/Search-For-Movie-Plots/blob/main/semantic_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Semantic Search - Wikipedia Question-Answer-Retrieval

This examples demonstrates the setup for Question-Answer-Retrieval.

You can input a query or a question. The script then uses semantic search
to find relevant passages in Simple English Wikipedia (as it is smaller and fits better in RAM).

As model, we use: nq-distilbert-base-v1

It was trained on the Natural Questions dataset, a dataset with real questions from Google Search together with annotated data from Wikipedia providing the answer. For the passages, we encode the Wikipedia article tile together with the individual text passages.

##Installing and importing packages and dependecies

In [1]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 2.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.3 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset
import json
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import time
import gzip
import os
import torch

if not torch.cuda.is_available():
  print("Warning: No GPU found. Please add GPU to your notebook")


##Importing data

In [4]:
from datasets import load_dataset
ds = load_dataset("Coder-Dragon/wikipedia-movies", split='train[:1000]')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

## Encoding Movies
So they will be searchable.

In [15]:
# We use the Bi-Encoder to encode all movies, so that we can use it with sematic search
model_name = 'nq-distilbert-base-v1'
bi_encoder = SentenceTransformer(model_name)

# create movies[] as [title, plot]
movies = []
for title, plot in zip(ds['Title'], ds['Plot']):
  movies.append([title, plot])



In [16]:
# examine movies[]
print(len(movies), "\n")
movies[:3]

1000 



[['Kansas Saloon Smashers',
  "A bartender is working at a saloon, serving drinks to customers. After he fills a stereotypically Irish man's bucket with beer, Carrie Nation and her followers burst inside. They assault the Irish man, pulling his hat over his eyes and then dumping the beer over his head. The group then begin wrecking the bar, smashing the fixtures, mirrors, and breaking the cash register. The bartender then sprays seltzer water in Nation's face before a group of policemen appear and order everybody to leave.[1]"],
 ['Love by the Light of the Moon',
  "The moon, painted with a smiling face hangs over a park at night. A young couple walking past a fence learn on a railing and look up. The moon smiles. They embrace, and the moon's smile gets bigger. They then sit down on a bench by a tree. The moon's view is blocked, causing him to frown. In the last scene, the man fans the woman with his hat because the moon has left the sky and is perched over her shoulder to see everythi

In [18]:
# create corpus embeddings
corpus_embeddings = bi_encoder.encode(movies, convert_to_tensor=True, show_progress_bar=True)

corpus_embeddings

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

tensor([[ 0.4242,  0.3535, -0.4906,  ..., -0.4527, -0.7219, -0.3250],
        [ 0.4416,  0.4091,  0.6788,  ..., -0.3532, -0.1965, -0.0579],
        [ 0.4367,  0.0597, -0.0720,  ...,  0.0901,  0.1567,  0.0581],
        ...,
        [ 0.1280, -0.4610, -0.6371,  ...,  0.3196, -0.1572,  0.8058],
        [ 0.1239,  0.1953, -0.4919,  ...,  0.2174,  0.0470, -0.1063],
        [-0.2902, -0.0538, -0.3584,  ..., -0.0562, -0.3267, -0.3031]],
       device='cuda:0')

##Query Encoding

In [19]:
def search(query, top_k = 5):
    # Encode the query using the bi-encoder and find potentially relevant movies
    start_time = time.time()
    query_embedding = bi_encoder.encode(query, convert_to_tensor=True)

    # use util.semantic_search to return cosine similarity and top k
    hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=top_k)
    hits = hits[0]  # Get the hits for the first query
    end_time = time.time()

    # Output of top-k hits
    print("Query: ", query)
    print("Results (after {:.3f} seconds):".format(end_time - start_time))
    for hit in hits:
      print("\t{:.3f}\t{}".format(hit['score'], movies[hit['corpus_id']]))

###Query 1

In [21]:
search("Documentaries showcasing indigenous peoples' survival and daily life in Arctic regions")

Query:  Documentaries showcasing indigenous peoples' survival and daily life in Arctic regions
Results (after 0.016 seconds):
	0.457	['Nanook of the North', 'The documentary follows the lives of an Inuk, Nanook, and his family as they travel, search for food, and trade in the Ungava Peninsula of northern Quebec, Canada. Nanook; his wife, Nyla; and their family are introduced as fearless heroes who endure rigors no other race could survive. The audience sees Nanook, often with his family, hunt a walrus, build an igloo, go about his day, and perform other tasks.']
	0.264	['David Copperfield', '"David Copperfield consists of three reels and as three separate films, released in three consecutive weeks, with three different titles: The Early Life of David Copperfield, Little Em\'ly and David Copperfield and The Loves of David Copperfield.[4]']
	0.251	['Straight Shooting', 'At the end of the 19th century in the Far West, a farmer is fighting for his right to plough the plains. In order to ex

Ground Truth Movie: 'Nanook of the North'

Recall = 1

RR = 1

###Query 2

In [22]:
search("Western romance")

Query:  Western romance
Results (after 0.017 seconds):
	0.305	['The Great Gatsby', "An adaptation of F. Scott Fitzgerald's Long Island-set novel, where Midwesterner Nick Carraway is lured into the lavish world of his neighbor, Jay Gatsby. Soon enough, however, Carraway will see through the cracks of Gatsby's nouveau riche existence, where obsession, madness, and tragedy await."]
	0.295	["Youth's Endearing Charm", 'The film is about a court case and embezzlement.']
	0.283	['Frankenstein', 'Described as "a liberal adaptation of Mrs. Shelley\'s famous story", the plot description in the Edison Kinetogram was:[3]']
	0.280	['High Society Blues', 'A new country family comes to live among established wealthy neighbors.']
	0.277	['Romance', 'As described in a film publication,[2] a youth (Arthur Rankin) in the prologue seeks advice from his grandfather (Sydney), who then recalls a romance of his own youth which is then shown as a flashback. A priest (Sydney) is in love with an Italian opera si

Ground Truth Movie: 'The Lucky Horseshoe'

Recall = 0

RR = 0

###Query 3

In [23]:
search("Silent film about a Parisian star moving to Egypt, leaving her husband \
for a baron, and later reconciling after finding her family in poverty in Cairo.")

Query:  Silent film about a Parisian star moving to Egypt, leaving her husband for a baron, and later reconciling after finding her family in poverty in Cairo.
Results (after 0.013 seconds):
	0.449	['Married in Hollywood', "A showgirl, part of a troupe, tours Europe where she falls in love with a Balkan prince. The prince's parents disapprove and attempt to put a stop to the romance. A revolution occurs and the prince and the showgirl elope to Hollywood."]
	0.394	['The King on Main Street', 'King Serge IV of Molvania (Menjou) comes to a small American town, and falls in love with one of its residents, Mary Young (Love).[3][4]']
	0.392	['Sahara', "Silent film femme fatale, Louise Glaum, portrays the role of Mignon, a Parisian music hall celebrity. Mignon marries a young American civil engineer, John Stanley, portrayed by Matt Moore. Stanley is transferred to Egypt to work on an engineering project in the Sahara. Mignon and her son, portrayed by Pat Moore, join Stanley in the desert.[3][

Ground Truth Movie: 'Sahara'

Recall = 0

RR = 1/3 (ranked 3rd)

###Query 4

In [24]:
search("Comedy film, office disguises, boss's daughter, elopement.")

Query:  Comedy film, office disguises, boss's daughter, elopement.
Results (after 0.013 seconds):
	0.403	["Youth's Endearing Charm", 'The film is about a court case and embezzlement.']
	0.400	['Dressed to Kill', 'The gang of a mob boss grow suspicious of his new girlfriend. She\'s a beautiful young girl and they don\'t believe she would actually associate with the mob and wonder if she\'s really a police "plant". The mobsters dress nattily to not appear "out of place" in the ritzy neighborhoods prior to a heist.']
	0.398	['The Boy Friend', 'Comedy about a small-town girl unhappy with her family, and a boy trying to please her by throwing a big party.']
	0.390	['A Little Journey', 'A girl travelling by train to meet her boyfriend meets another young man and falls in love with him.']
	0.373	['This Is Heaven', "Vilma Banky portrays a newly arrived Hungarian immigrant who learns to accustom herself to the new and strange life she finds in New York. The story gave Miss Banky moments of come

Ground Truth Movie: 'Ask Father'

Recall = 0

RR = 0

###Query 5

In [25]:
search("Lost film, Cleopatra charms Caesar, plots world rule, treasures from \
mummy, revels with Antony, tragic end with serpent in Alexandria.")

Query:  Lost film, Cleopatra charms Caesar, plots world rule, treasures from mummy, revels with Antony, tragic end with serpent in Alexandria.
Results (after 0.015 seconds):
	0.537	['Cleopatra', "Because the film has been lost, the following summary is reconstructed from a description in a contemporary film magazine.\r\nCleopatra (Bara), the Siren of Egypt, by a clever ruse reaches Caesar (Leiber) and he falls victim to her charms. They plan to rule the world together, but then Caesar falls. Cleopatra's life is desired by the church, as the wanton woman's rule has become intolerable. Pharon (Roscoe), a high priest, is given a sacred dagger to take her life. He gives her his love instead and, when she is in need of some money, leads her to the tomb of his ancestors, where she tears the treasure from the breast of the mummy. With this wealth she goes to Rome to meet Antony (Hall). He leaves the affairs of state and travels to Alexandria with her, where they revel. Antony is recalled to R

Ground Truth Movie: 'Cleopatra'

Recall = 1

RR = 1

###Query 6

In [26]:
search("Denis Gage Deane-Tanner")

Query:  Denis Gage Deane-Tanner
Results (after 0.013 seconds):
	0.365	['Souls for Sale', 'Remember "Mem" Steddon (Eleanor Boardman) marries Owen Scudder (Lew Cody) after a whirlwind courtship. However, on their wedding night, she has a change of heart. When the train taking them to Los Angeles stops for water, she impulsively and secretly gets off in the middle of the desert. Strangely, when Scudder realizes she is gone, he does not have the train stopped.\r\nMem sets off in search of civilization. Severely dehydrated, she sees an unusual sight: an Arab on a camel. It turns out to be actor Tom Holby (Frank Mayo); she has stumbled upon a film being shot on location. When she recuperates, she is given a role as an extra. Both Holby and director Frank Claymore (Richard Dix) are attracted to her. However, when filming ends, she does not follow the troupe back to Hollywood, but rather gets a job at a desert inn.\r\nMeanwhile, Scudder is recognized and arrested at the train station. He turns

Ground Truth Movie: 'Captain Alvarez'

Recall = 0


RR = 0

##Evaluation Metrics
**Recall@1**

number of true positives divided by the number of true positives + false negatives (total actual positives). This is a measure of how good the model is at retrieving relevant instances out of all the relevant instances.

Here we will be measuring if our  "ground truth movie" (1) is  the first movie returned: 1 = yes ;  0 = no

**Mean Reciprocal Rank (MRR)** takes the avverage of the highest ranks across queries, for example if a movie ranks 3rd, its reciprocal rank is 1/3, this number is then averaged into the total of all rankings considered.

A MMR is measured from 0 to 1, where 1 means that the relevant item is the top result. A higher MMR means better performance.

##Analysis

Only 2 out of 6 queries returned the Ground Truth movie as the top returned result. = .33

Adding the Reciprocal Rank values (out of 5) for each query we get .38

(1, 1, 1/3, 0, 0, 0) / 6 = .38


**recall = .33**

**MMR = .38**

Both of these are quite low scores.



*   What type of queries tend to do Well? Which not so well?
    
    - Queries that were more descriptive (and unique in that description) did better. Queries 1, 3, and 5. Q1 was descriptive and straightforward, specifying genre and film contents. Q3 very described the plot line very thoroughly. Q5's plot and topic are very clear and this may have more to do with the subject matter of the movie than the query structure. Q4 was formatted very similarly to Q5 but did not return as much success, the description was quite vague. Q3 is also very vague. There are thousands of western romances.
*   For the queries that the model didn't perform well, what could be two alternative approaches?

    - Beyond being more specific and descriptive in query and using more keywords, and structuring queries in a more natural way,  maybe expanding criteria to include details like cast, shooting location, maybe even descriptions of actors that could return similar films that an actor has starred in. "What about that murder mystery that is set in Egypt on the nile starring the actress that played wonder woman?" Gal Gadot.

